# Building a sequential multi-agent travel assistant with CrewAI

This notebook demonstrates how to create a travel assistant using multi-agent collaboration with the CrewAI framework. Specifically, you will use this assistant to generate articles on the top activities and attractions for a specific location. 

## Structure of the travel assistant
This notebook provides step-by-step guidance for creating a travel assistant consisting of the following agents:

1. Researcher Agent - is responsible for gathering information on the top 5 attractions and activities in a specific location
2. Content Writer Agent - is responsible for writing an engaging article based on the information of the top 5 attractions
3. Editor Agent - is responsible for improving the flow and language use within the article

## (Optional) Prerequisite knowledge
This section provides general information on agents, multi-agent collaboration, and other topics. If you are comfortable with these concepts, feel free to skip reading this section. 

### What is an LLM and what do the parameters mean?
[Large language models](https://aws.amazon.com/what-is/large-language-model/), also known as LLMs, are very large deep learning models that are pre-trained on vast amounts of data. LLMs are incredibly flexible. One model can perform completely different tasks such as answering questions, summarizing documents, translating languages and completing sentences. While not perfect, LLMs are demonstrating a remarkable ability to make predictions based on a relatively small number of prompts or inputs.

You can interact with LLMs through prompts. A prompt is a natural language text that requests the LLM to perform a specific task. Even though LLM attempts to mimic humans, it requires detailed instructions to create high-quality and relevant output. In [prompt engineering](https://aws.amazon.com/what-is/prompt-engineering/), you choose the most appropriate formats, phrases, words, and symbols that guide the AI to interact with your users more meaningfully. Prompt engineers use creativity plus trial and error to create a collection of input texts, so an application's model works as expected.

In addition to prompts, you can influence LLM behavior through [inference parameters](https://docs.aws.amazon.com/bedrock/latest/userguide/inference-parameters.html). Every model has different inference parameters, but a common parameter is `temperature`. It affects the shape of the probability distribution for the predicted output and influences the likelihood of the model selecting lower-probability outputs. A lower temperature leads to more deterministic responses which is useful for classification and structured output. A higher temperature leads to more random responses which is useful for use cases requiring natural, human-sounding, and creative output. 

### What is an agent and why should I use build one?
An [artificial intelligence (AI) agent](https://aws.amazon.com/what-is/ai-agents/) is a software program that can interact with its environment, collect data, and use the data to perform self-determined tasks to meet predetermined goals. Humans set goals, but an AI agent independently chooses the best actions it needs to perform to achieve those goals. Alternatively, the term agent is used to refer to an LLM workflow which is orchestrated to follow pre-defined tasks and paths. Agents can be given access to tools, such as calculators, APIs, or databases, to complete the tasks they have been given. 

### Why should I use multiple agents to build a solution?
Large language model (LLM) based AI agents that have been specialized for specific tasks have demonstrated great problem-solving capabilities. By combining the reasoning power of multiple intelligent specialized agents, multi-agent collaboration has emerged as a powerful approach to tackle more intricate, multistep workflows. Multi-agent collaboration offers several key advantages over single-agent approaches, primarily stemming from distributed problem-solving and specialization.

Distributed problem-solving refers to the ability to break down complex tasks into smaller subtasks that can be handled by specialized agents. By breaking down tasks, each agent can focus on a specific aspect of the problem, leading to more efficient and effective problem-solving. For example, a travel planning problem can be decomposed into subtasks such as checking weather forecasts, finding available hotels, and selecting the best routes. The distributed aspect also contributes to the extensibility and robustness of the system. As the scope of a problem increases, we can simply add more agents to extend the capability of the system rather than try to optimize a monolithic agent packed with instructions and tools. On robustness, the system can be more resilient to failures because multiple agents can compensate for and even potentially correct errors produced by a single agent.

Specialization allows each agent to focus on a specific area within the problem domain. For example, in a network of agents working on software development, a coordinator agent can manage overall planning, a programming agent can generate correct code and test cases, and a code review agent can provide constructive feedback on the generated code. Each agent can be designed and customized to excel at a specific task. 
For developers building agents, this means the workload of designing and implementing an agentic system can be organically distributed, leading to faster development cycles and better quality. Within enterprises, often development teams have distributed expertise that is ideal for developing specialist agents. Such specialist agents can be further reused by other teams across the entire organization.

## Setup

Start by installing some of the required packages, including CrewAI for building multi-agent solutions, Langchain for pre-built tool components, and Tavily for its web search API.

In [ ]:
!pip install -r crewai-requirements.txt -qU

<div class="alert alert-block alert-info">
<b>Important:</b> restart the kernel before proceeding with the next cells.
</div>

Now you can import all of the packages you will need for this lab. This includes the packages described above, but also textwrap for keeping the prompts readable within the notebook, and pydantic for validating input.

In [ ]:
from crewai import LLM, Agent, Crew, Task, Process
from textwrap import dedent
from crewai_tools import SerperDevTool

Follow the instructions in [this lab](https://catalog.us-east-1.prod.workshops.aws/workshops/eb7af74b-184e-47d2-9277-4f59b4c91df5/en-US/2-llm-deployment-with-aws/2-sagemaker) to create a SageMaker real-time endpoint hosting the [DeepSeek R1 Distill-Llama 8B](https://huggingface.co/deepseek-ai/DeepSeek-R1) model. Although you will use this specific model for this lab, CrewAI also supports other models hosted through [Amazon SageMaker](https://docs.crewai.com/concepts/llms#amazon-sagemaker) or [Amazon Bedrock](https://docs.crewai.com/concepts/llms#aws-bedrock). While the SageMaker endpoint is being created, you can copy the name of the endpoint and replace the text `INSERT ENDPOINT NAME` in the code below with the endpoint name.

When setting up an LLM model to use with CrewAI, you can also set model parameters. In this case, we set `temperature` to 0.7 to balance creativity with accuracy. We also set the `max_tokens` to 4096 to allow for longer responses.

In [ ]:
reasoning_llm = LLM(
    model="sagemaker/INSERT ENDPOINT NAME",
    temperature=0.7, max_tokens=4*1024,
)

## Create your first agent

Let's start by creating the first agent - the researcher agent. This agent will be responsible for gathering information on the top 5 attractions and activities for a specific location. Start simple by creating an agent which relies on the LLM's internal knowledge of various locations around the world. At a later stage, this agent can be extended by providing access to a web search tool. 

CrewAI Agents support various [attributes](https://docs.crewai.com/concepts/agents#agent-attributes), but we will start with:
* `role` - defines the agent's function within the crew
* `goal` - defines the objective which guides the agent's behavior
* `backstory` - provides additional context to enrich interactions
* `llm` - the LLM which processes the prompt and generates the output
* `allow_delegation` - allow the agent to delegate tasks to other agents
* `verbose` - enables detailed logs for debugging
* `max_iter` - maximum iterations the model can use to produce output

In [ ]:
researcher_agent = Agent(
    role="Travel Researcher",
    goal="Research and compile interesting activities and attractions for a given location",
    backstory=dedent(
        """You are an experienced travel researcher with a knack for 
        discovering both popular attractions and hidden gems in any 
        location. Your expertise lies in gathering comprehensive 
        information about various activities, their historical 
        significance, and practical details for visitors.
        """
    ),
    allow_delegation=False,
    verbose=True,
    llm=reasoning_llm,
    max_iter=4
)

Next, you will create a task. In CrewAI, a task is a specific action which can be completed by one or more agents. In this case, we define a research task which requires an agent to research at least 5 attractions and activities for a specific location.

CrewAI Tasks support various [attributes](https://docs.crewai.com/concepts/tasks#task-attributes), but we will start with:
* `description` - a concise statement of how the task should be completed
* `expected_output` - a description of the what the output should look like
* `agent` - the agent which will complete the task

In [ ]:
research_task = Task(
    description=dedent(
        """Research and compile a list of at least 5 interesting 
        activities and attractions in {location}. Include a mix of 
        popular tourist spots and lesser-known local favorites. For 
        each item, provide:
        1. Name of the attraction/activity
        2. Brief description (2-3 sentences)
        3. Why it's worth visiting
        Your final answer should be a structured list of these items.
        """
    ),
    agent=researcher_agent,
    expected_output="Structured list of 5+ attractions/activities",
)

Now you can create your first crew. A crew is a collection of agents which work together to complete tasks. This crew is particularly small, having only one agent, but you will expand it later.

CrewAI Crews support various [attributes](https://docs.crewai.com/concepts/crews#crew-attributes), but we will start with:
* `agents` - a list of agents to be included in the crew
* `tasks` - a list of tasks to be completed by the agents
* `verbose` - enables detailed logs for debugging

In [ ]:
crew = Crew(
    agents=[researcher_agent],
    tasks=[research_task],
    verbose=True,
)

Now you can test your first crew! Since the current version of the researcher agent relies on the LLM's internal knowledge, choose a well-known location such as Amsterdam. The model has likely learned about Amsterdam in its training data. 

In [ ]:
inputs = {
    'location': 'Amsterdam, Netherlands'
}
listicle_result = crew.kickoff(inputs=inputs)
print(listicle_result)

## Add more agents to the crew

A crew of one agent is not very interesting, so let's add a second agent to the crew. Using the same techniques as above, create a second agent which specializes in writing informative top 5 travel listicles (a type of article which contains a list). 

In [ ]:
content_writer_agent = Agent(
    role="Travel Content Writer",
    goal="Create engaging and informative content for the top 5 listicle",
    backstory=dedent(
        """You are a skilled travel writer with a flair for creating 
        captivating content. Your writing style is engaging, 
        informative, and tailored to inspire readers to explore new 
        destinations. You excel at crafting concise yet compelling 
        descriptions of attractions and activities."""
    ),
    allow_delegation=False,
    verbose=True,
    llm=reasoning_llm,
    max_iter=4
)

write_task = Task(
    description=dedent(
        """Create an engaging top 5 listicle article about things to 
        do in {location}. Use the research provided to:
        1. Write a catchy title and introduction (100-150 words)
        2. Select and write about the top 5 activities/attractions
        3. For each item, write 2-3 paragraphs (100-150 words total)
        4. Include a brief conclusion (50-75 words)

        Ensure the content is engaging, informative, and inspiring. 
        Your final answer should be the complete listicle article.
        """
    ),
    agent=content_writer_agent,
    expected_output="Complete top 5 listicle article",
)

Of course, professional media companies employ editors to improve the quality of articles. By taking a second look at a good article, we can make it great. So let's create a third agent which specializes in editing existing articles to improve the flow of the story and improve readability. 

In [ ]:
editor_agent = Agent(
    role="Content Editor",
    goal="Ensure the listicle is well-structured, engaging, and error-free",
    backstory=dedent(
        """You are a meticulous editor with years of experience in 
        travel content. Your keen eye for detail helps polish articles 
        to perfection. You focus on improving flow, maintaining 
        consistency, and enhancing the overall readability of the 
        content while ensuring it appeals to the target audience."""
    ),
    allow_delegation=False,
    verbose=True,
    llm=reasoning_llm,
    max_iter=4
)

edit_task = Task(
    description=dedent(
        """Review and edit the top 5 listicle article about things to 
        do in {location}. Focus on:
        1. Improving the overall structure and flow
        2. Enhancing the engagement factor of the content
        3. Ensuring consistency in tone and style
        4. Correcting any grammatical or spelling errors

        Do not change the content itself. Only edit it for higher quality.
        Your final answer should be the polished, publication-ready 
        version of the article.
        """
    ),
    agent=editor_agent,
    expected_output="Edited and polished listicle article about {location}",
)

After creating your second and third agent, you will need to add them both to your crew. Tasks can be executed sequentially (i.e. always in the order which they are defined) or hierarchically (i.e. tasks are assigned based on agent roles). In this case, the process for writing a travel article is always the same, so we will use sequential.

In [ ]:
crew = Crew(
    agents=[researcher_agent, content_writer_agent, editor_agent],
    tasks=[research_task, write_task, edit_task],
    process=Process.sequential,
    verbose=True,
)

Next, test your crew again using the same location as before, such as Amsterdam. You should see the agents executing the tasks in order, resulting in an article about the top 5 attractions for your location.

In [ ]:
inputs = {
    'location': 'Amsterdam, Netherlands'
}
listicle_result = crew.kickoff(inputs=inputs)
print(listicle_result)

What happens if you are looking for travel advice for a less well-known location? A model is unlikely to know much about a small village like Hoenderloo, Netherlands (population of ~1,500), even though it has a lot to offer. In this case, the model will likely give recommendations outside of the location itself, or it may even hallucinate attractions which don't exist.

In [ ]:
inputs = {
    'location': 'Hoenderloo, Netherlands'
}
listicle_result = crew.kickoff(inputs=inputs)
print(listicle_result)

## Enhance agent capabilities using tools

Let's extend the capabilities of the research agent by giving it access to a web search tool so it can search for relevant information about a less well-known location. 

There are many API search tools available. CrewAI has a [built-in tool](https://docs.crewai.com/concepts/tools) for [Google Serper Search](https://docs.crewai.com/tools/serperdevtool), which offers a limited number of free queries. To complete this section, you will need to [register](https://serper.dev/) and fetch an API key for this service. Copy the API key and provide it when prompted in the next cell. This will store your API key as an environment variable.

Now you can instantiate the Serper Dev search tool. The tool can perform specific searches for news, scholarly articles, images, and more, but in this case you will use the tool to perform a generic internet search. This can be specified using the `search_url` parameter. It is recommended to set the maximum number of search results using `n_results` parameter.

In [ ]:
from getpass import getpass
import os
from dotenv import load_dotenv

# Make sure the Serper API Key is configured
load_dotenv()
if not os.getenv("SERPER_API_KEY"):
    os.environ["SERPER_API_KEY"] = getpass("Please enter your Serper API key: ")

search_tool = SerperDevTool(
    search_url="https://google.serper.dev/search",
    n_results=6,
)

<div class="alert alert-block alert-warning">
<h3><b>Having trouble with Serper API Key?</b></h3>
Run the next few cells to create a custom Tool to use DuckDuckGo to perform searches. Less stable than Serper.<b>If you've managed to get a Serper API Key</b>, ignore the next two cells.
</div>

In [ ]:
%pip install duckduckgo-search -qU

In [ ]:
from crewai.tools import tool
from duckduckgo_search import DDGS

@tool('DuckDuckGoSearch')
def search_tool(search_query: str):
    """Search the web for information on a given topic"""
    return DDGS().text(search_query, max_results=5)

DeepSeek R1 does not natively support function calling or structured outputs ([source](https://github.com/deepseek-ai/DeepSeek-R1/issues/9#issuecomment-2604747754)). With careful prompting it is possible to call a tool with DeepSeek R1, but we recommend choosing the right model for the right job. For example, **Meta Llama 3.1 70B Instruct** is better at tool calling, so we will use it for the research agent. Although you can deploy Meta Llama models on SageMaker endpoints, let's try using Llama via Amazon Bedrock serverless endpoints instead.

In [ ]:
function_calling_llm = LLM(
    model="bedrock/invoke/meta.llama3-1-70b-instruct-v1:0",
    temperature=0, max_gen_len=5*1024
)

Recreate the researcher agent to include the tool. In addition to adding the tool in the `tools` parameter, you will want to adjust the `goal` and `backstory` to inform the agent that it has access to the tool. And change the `llm` to point to the Llama model.

In [ ]:
researcher_agent = Agent(
    role="Travel Researcher",
    goal="Research interesting activities and attractions for a given location using the web search tool once.",
    backstory=dedent(
        """You are an experienced travel researcher with a knack for 
        discovering both popular attractions and hidden gems in any 
        location. Your expertise lies in running one web search for 
        comprehensive information about various activities, their 
        historical significance, and practical details for visitors.
        Use the web search tool to find the information you need.
        """
    ),
    tools=[search_tool],
    allow_delegation=False,
    verbose=True,
    llm=function_calling_llm,
    max_iter=4
)

You can also re-create the task and edit the prompt slightly to ensure that the model knows it can use the web search tool.

In [ ]:
research_task = Task(
    description=dedent(
        """You have access to a web search tool to research and compile 
        a list of at least 5 interesting activities and attractions in 
        {location}. Include a mix of popular tourist spots and lesser-known 
        local favorites. For each item, provide:
        1. Name of the attraction/activity
        2. Brief description (2-3 sentences)
        3. Why it's worth visiting
        Your final answer should be a structured list of these items.
        """
    ),
    agent=researcher_agent,
    expected_output="Structured list of 5+ attractions/activities",
)

Recreate the crew with your new research agent. No adjustment is needed for this code.

In [ ]:
crew = Crew(
    agents=[researcher_agent, content_writer_agent, editor_agent],
    tasks=[research_task, write_task, edit_task],
    process=Process.sequential,
    verbose=True,
)

Now let's test the new crew of agents by asking for travel advice for Hoenderloo. Feel free to test with your location of choice.

In [ ]:
inputs = {
    'location': 'Hoenderloo, Netherlands'
}
listicle_result = crew.kickoff(inputs=inputs)
print(listicle_result)